In [ ]:
import gdown
import shutil
from astrohack import panel
import os

In [ ]:
# Download VLA Holography with known panel deviationsurl = "https://drive.google.com/u/1/uc?id=10fXyut_UHPUjIuaaEy6-m6wcycZHit2v&export=download"
datafolder = "./paneldata/"
os.makedirs(name=datafolder, exist_ok=True)
panelzip = datafolder+"panel.zip"
if not os.path.exists(panelzip):
    url = "https://drive.google.com/u/1/uc?id=10fXyut_UHPUjIuaaEy6-m6wcycZHit2v&export=download"
    gdown.download(url, panelzip)
shutil.unpack_archive(filename=panelzip, extract_dir=datafolder)

In [ ]:
# Declare a path to the Amplitude and Deviation images produced by AIPS
npoints = 43
telescope = 'VLA'
# Create a new Antenna_Surface
vla = panel.Antenna_Surface(datafolder+'amp.fits',datafolder+'dev.fits',
                            npoints,telescope)

In [ ]:
%%time
# Compile the list of points inside each panel, this is the longest step
vla.compile_panel_points()

In [ ]:
%%time 
# Fit surface to get the parameters for each panel
vla.fit_surface()

In [ ]:
%%time
# Apply corrections to create a new antenna surface
vla.correct_surface()

In [ ]:
# Compute the gains at a wavelenght as well as the antenna surface RMSes
wave = 1 # Unit may be cm or mm still not clear
ingains, ougains = vla.gains(wave)
inrms, ourms     = vla.get_rms()
print("Gains before correction: Real: {0:7.3} dB, Theoretical: {1:7.3} dB".format(*ingains))
print("RMS before correction: {0:7.3} mm".format(inrms))
print()
print("Gains after correction: Real: {0:7.3} dB, Theoretical: {1:7.3} dB".format(*ougains))
print("RMS after correction: {0:7.3} mm".format(ourms))

In [ ]:
vla.plot_surface()

In [ ]:
# Export products of the panel fitting
vla.export_screw_adjustments(datafolder+'screw-adjustments.txt')
vla.export_corrected(datafolder+'corrected-surface.fits')